In [2]:
from pathlib import Path
import pandas as pd
import os
import csv
import numpy as np
import requests
import time
import io
import yfinance as yf
import warnings
# Initial imports
#%tensorflow_version 1.x
import tensorflow
import mibian
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lstm
import psutil
import main_lstm
import main_rf
psutil.virtual_memory()

def create_spreads(data):
    #Sort Values and shift to set up collateral requirements
    data['DataDate'] = data.index
    data['DateKey']=data['DataDate'].astype('str')+data['ExpirationDate'].astype('str') 
    
    data['DataDate'] = data['DataDate'].apply(pd.Timestamp)
    data.groupby('DateKey') 
    #data.sort_values('DataDate') 
    data['Prev_Strike']=data['StrikePrice'].shift(1)
    data['Prev_AskPrice']=data['AskPrice'].shift(1)
    data['Prev_DateKey']=data['DateKey'].shift(1)
    data['Prev_xp']=data['xp'].shift(1)
      #Create nan values for the crossover days
    for idx, row in data.iterrows():
        if row['Prev_DateKey']!=row['DateKey']:
            row['Prev_Strike']=np.nan
            row['Prev_AskPrice']=np.nan
            row['Prev_xp']=np.nan
    data['spread_premium'] = data['AskPrice'].astype(float) - data['Prev_AskPrice'].astype(float)
    data['spread_xp'] = data['xp'].astype(float) - data['Prev_xp'].astype(float)
    data['spread_profit'] = data['spread_premium'].astype(float) - data['spread_xp'].astype(float)
    return data     
        # for i in range(len(data)):    
    
#     # Create nan values where the shifted value uses a different day
#     for i in data.index:
#         if data.loc[i, 'Prev_DateKey']!=data.loc[i, 'DateKey']:
#             data.loc[i, 'Prev_Strike']=np.nan
#             data.loc[i, 'Prev_AskPrice']=np.nan
            
#     return data

def buying_puts_blp(df):
    blc = df.loc[(df['bollinger_signal']== -1) & (df['Put signals'] == 1)]
    #blc['sclog'] = blc['log']
    #blc_total = round(blc['sclog'].sum() * -1)
    blc['log'] = blc['AskPrice'].astype(float) - blc['xp'].astype(float)

    blc_otm_xs = blc.loc[blc['otm_xs'] == True]
    blc_otm_xs = blc_otm_xs[['log']]
    blc_otm_xs['log'] = (blc_otm_xs['log'] * -1)
    blc_otm_xs.to_csv(f'a{symbol}buying_puts_blp_otm_xs.csv')
    return blc_otm_xs

    blc_otm_xm = blc.loc[blc['otm_xm'] == True]
    blc_otm_xm['log'] = round(blc_otm_xm['log'] * -1)
    blc_otm_xm = blc_otm_xm['log']
#    blc_otm_xm.to_csv(f'bearish_{symbol}buying_puts_blp_otm_xm.csv')

    blc_otm_xf = blc.loc[blc['otm_xf'] == True]
    blc_otm_xf['log'] = round(blc_otm_xf['log'] * -1)
    blc_otm_xf = blc_otm_xf['log']
    blc_otm_xf.to_csv(f'bearish_{symbol}buying_puts_blp_otm_xf.csv')
    
    blc_ctm_xs = blc.loc[blc['ctm_xs'] == True]
    blc_ctm_xs['log'] = round(blc_ctm_xs['log'] * -1)
    blc_ctm_xs = blc_ctm_xs['log']
    blc_ctm_xs.to_csv(f'bearish_{symbol}buying_puts_blp_ctm_xs.csv')
    
    blc_ctm_xf = blc.loc[blc['ctm_xf'] == True]
    blc_ctm_xf['log'] = round(blc_ctm_xf['log'] * -1)
    blc_ctm_xf = blc_ctm_xf['log']
    blc_ctm_xf.to_csv(f'bearish_{symbol}buying_puts_blp_ctm_xf.csv')
    
    blc_ctm_xm = blc.loc[blc['ctm_xm'] == True]
    blc_ctm_xm['log'] = round(blc_ctm_xm['log'] * -1)
    blc_ctm_xm = blc_ctm_xm['log']
    blc_ctm_xm.to_csv(f'bearish_{symbol}buying_puts_blp_ctm_xm.csv')
    #blc_ctm_xf['log'] = blc_ctm_xf['log']
    
    #blc_ctm_xf['log'] = round(blc_ctm_xf['log'])
    #blc_ctm_xf_total = blc_ctm_xf[:, 0].sum()

    print(f'blindly buying puts on bollinger and call signals would net:{blc_total}')
    #print(f'buying close to the money, expiring far calls on bollinger and call signals would net: {blc_ctm_xf_total}' )
    

def selling_put_spreads_blp(df):
    blc = df.loc[(df['bollinger_signal']== 1) & (df['Call signals'] == 1)]
    blc['log'] = blc['spread_profit']
    blc_total = round(blc['log'].sum())
    
    blc_otm_xs = blc.loc[blc['otm_xs'] == True]
    blc_otm_xs['spread_profit'] = round(blc_otm_xs['spread_profit'])
    blc_otm_xs = blc_otm_xs['spread_profit']
    blc_otm_xs.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_otm_xs.csv')
    
    blc_otm_xm = blc.loc[blc['otm_xm'] == True]
    blc_otm_xm['spread_profit'] = round(blc_otm_xm['spread_profit'])
    blc_otm_xm = blc_otm_xm['spread_profit']
    blc_otm_xm.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_otm_xm.csv')

    blc_otm_xf = blc.loc[blc['otm_xf'] == True]
    blc_otm_xf['spread_profit'] = round(blc_otm_xf['spread_profit'])
    blc_otm_xf = blc_otm_xf['spread_profit']
    blc_otm_xf.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_otm_xf.csv')
    
    blc_ctm_xs = blc.loc[blc['ctm_xs'] == True]
    blc_ctm_xs['spread_profit'] = round(blc_ctm_xs['spread_profit'])
    blc_ctm_xs = blc_ctm_xs['spread_profit']
    blc_ctm_xs.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_ctm_xs.csv')
    
    blc_ctm_xf = blc.loc[blc['ctm_xf'] == True]
    blc_ctm_xf['spread_profit'] = round(blc_ctm_xf['spread_profit'])
    blc_ctm_xf = blc_ctm_xf['spread_profit']
    blc_ctm_xf.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_ctm_xf.csv')
    
    blc_ctm_xm = blc.loc[blc['ctm_xm'] == True]
    blc_ctm_xm['spread_profit'] = round(blc_ctm_xm['spread_profit'])
    blc_ctm_xm = blc_ctm_xm['spread_profit']
    blc_ctm_xm.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_ctm_xm.csv')
    #blc_ctm_xf['spread_profit'] = blc_ctm_xf['spread_profit']
    
    #blc_ctm_xf['spread_profit'] = round(blc_ctm_xf['spread_profit'])
    #blc_ctm_xf_total = blc_ctm_xf[:, 0].sum()

    print(f'blindly selling put spreads on bollinger and call signals would net:{blc_total}')
    #print(f'buying close to the money, expiring far calls on bollinger and call signals would net: {blc_ctm_xf_total}' )
    

def selling_puts_blp(df):
    blc = df.loc[(df['bollinger_signal']== 1) & (df['Call signals'] == 1)]
    blc['log'] = blc['AskPrice'].astype(float) - blc['xp'].astype(float)
    
    blc_otm_xs = blc.loc[blc['otm_xs'] == True]
    blc_otm_xs['log'] = round(blc_otm_xs['log'])
    blc_otm_xs = blc_otm_xs['log']
    blc_otm_xs.to_csv(f'bullish_{symbol}selling_puts_blc_otm_xs.csv')
    
    blc_otm_xm = blc.loc[blc['otm_xm'] == True]
    blc_otm_xm['log'] = round(blc_otm_xm['log'])
    blc_otm_xm = blc_otm_xm['log']
    blc_otm_xm.to_csv(f'bullish_{symbol}selling_puts_blc_otm_xm.csv')

    blc_otm_xf = blc.loc[blc['otm_xf'] == True]
    blc_otm_xf['log'] = round(blc_otm_xf['log'])
    blc_otm_xf = blc_otm_xf['log']
    blc_otm_xf.to_csv(f'bullish_{symbol}selling_puts_blc_otm_xf.csv')
    
    blc_ctm_xs = blc.loc[blc['ctm_xs'] == True]
    blc_ctm_xs['log'] = round(blc_ctm_xs['log'])
    blc_ctm_xs = blc_ctm_xs['log']
    blc_ctm_xs.to_csv(f'bullish_{symbol}selling_puts_blc_ctm_xs.csv')
    
    blc_ctm_xf = blc.loc[blc['ctm_xf'] == True]
    blc_ctm_xf['log'] = round(blc_ctm_xf['log'])
    blc_ctm_xf = blc_ctm_xf['log']
    blc_ctm_xf.to_csv(f'bullish_{symbol}selling_puts_blc_ctm_xf.csv')
    
    blc_ctm_xm = blc.loc[blc['ctm_xm'] == True]
    blc_ctm_xm['log'] = round(blc_ctm_xm['log'])
    blc_ctm_xm = blc_ctm_xm['log']
    blc_ctm_xm.to_csv(f'bullish_{symbol}selling_puts_blc_ctm_xm.csv')
    #blc_ctm_xf['log'] = blc_ctm_xf['log']
    
    #blc_ctm_xf['log'] = round(blc_ctm_xf['log'])
    #blc_ctm_xf_total = blc_ctm_xf[:, 0].sum()

    print(f'blindly selling puts on bollinger and call signals would net:{blc_total}')
    #print(f'buying close to the money, expiring far calls on bollinger and call signals would net: {blc_ctm_xf_total}' )
    

def test_puts(df):
    buying_puts_blp(df)
    selling_put_spreads_blp(df)
    selling_puts_blp(df)

In [ ]:
#tech: FB, GOOG, MSFT,TSLA, AAPL
#etf: MTUM, NDAQ, SPY, VOO
#food: CPG, PG, SBUX
#finance: JPM, GS
#materials: BHP, RIO, VALE
#media: DIS, NFLX, TWTR
#military: BA, LMT,

sector = 'tech'
symbol = 'TSLA'
df = pd.read_pickle(f'aaafinally_{sector}_{symbol}_puts.pickle')
test_puts(df)